In [5]:
import json
import pandas
import pathlib
import pydash
import requests
import time
import tqdm
import uuid

data_path = pathlib.Path.cwd() / 'sight_and_sound.json'

if not data_path.exists():

    index = requests.get('https://www.bfi.org.uk/sight-and-sound/greatest-films-all-time/all-voters').text
    index = index.split('<script type="text/javascript">var initialPageState = ')[1].split('</script>')[0]
    index = pydash.get(json.loads(index), 'componentState.allVoters')

    data = list()

    for x in tqdm.tqdm(index[:2]):

        time.sleep(4)

        voter = {k:v for k,v in x.items() if k in ['firstname', 'surname', 'type', 'country', 'url']}
        voter['country'] = [{'country':x.strip()} for x in voter['country'].split('/')]

        votes = pandas.read_html('https://www.bfi.org.uk'+voter['url'], encoding='utf8')[0].to_dict('records')
        for y in votes:
            y['Director'] = [{'director':x.strip()} for x in str(y['Director']).split(',')]

        voter['votes'] = votes
        data.append(voter)

    with open(data_path, 'w') as write_data:
        json.dump(data, write_data, ensure_ascii=False, indent=4)
else:
    pass 

100%|██████████| 2/2 [00:08<00:00,  4.20s/it]
